## Script for all analyses relating to leatherback KDE analyses

   #### Includes data processing, KD generation, quantiles

### Data preparation

Upload the data file. Use the file: "[turtle_name]-[turtleID#]-1-FastGPS.csv"

In [ ]:
arcpy.management.XYTableToPoint(
    in_table="Queen Haevo-237820-1-FastGPS.csv",
    out_feature_class=r"\\Mac\Home\Documents\ArcGIS\Projects\Leatherback_KDEs\Leatherback_KDEs.gdb\QueenHaevo237820",
    x_field="Longitude",
    y_field="Latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)


Delete surplus fields from base Feature Layer using FastlocGPS.csv files

In [ ]:
arcpy.management.DeleteField(
    in_table="QueenHaevo237820", #replace 
    drop_field="Name;Day;Time;Hauled_Out;Latitude;Longitude",
    method="KEEP_FIELDS")